In [10]:
import pandas
import numpy
from numpy import std, correlate, sqrt
from scipy.stats import pearsonr
import statsmodels.api as sm
%run Table_Summary.ipynb

### Preparing data

In [11]:
filepath = 'D:/DEA/Data/Original_data.txt'
data = pandas.read_csv(filepath,sep='\t')
data.rename(columns={'origi':'Id'},inplace=True)
print(data.shape,'from 2019_05_25_Results.xlxs')
print(data.Year.unique(),'Years')
print(data.Id.nunique(),'Number of firms')
filepath = 'D:/DEA/Data/Database_secondreviewJIBS.dta' #
data1 = pandas.read_stata(filepath)
data1.rename(columns={'id':'Id','year':'Year','country':'Country'},inplace=True)
data1 = data1[['Id','Year','Country']]
data1['Year'] = pandas.to_numeric(data1.Year.astype(str).str[0:4])
print(data1.shape,'to get country')
data = pandas.merge(left=data,right=data1,how='left',on=['Year','Id'])
print(data.Country.unique())
filepath = 'D:/DEA/Data/2019_05_25_Results_updated.txt'
data2 = pandas.read_csv(filepath,sep='\t')
print(data2.shape,'correction by Elio of market commonality')
data2.rename(columns={'origi':'Id'},inplace=True)
data2 = data2[['Id','Year','Market_commonality']]
data.drop('Market_commonality',axis=1,inplace = True)
data = pandas.merge(left=data,right=data2,how='inner',on=['Year','Id'])
print(data.Non_RD_alliances.sum(),'Total number of Non-RD alliances')
print(data.RD_alliances.sum(),'Total number of RD alliances')

(1232, 33) from 2019_05_25_Results.xlxs
[1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
 2005 2006 2007 2008 2009 2010 2011 2012] Years
56 Number of firms
(1539, 3) to get country
[nan 'USA' 'Switzerland' 'Netherlands' 'Germany' 'South Africa' 'Japan'
 'United Kingdom' 'France' 'Australia' 'Ireland' 'Spain' 'Denmark'
 'Sweden']
(1232, 34) correction by Elio of market commonality
4089.0 Total number of Non-RD alliances
7676.0 Total number of RD alliances


### Creating variables

In [12]:
df = data.copy()
df = df.sort_values(by=['Id','Year'],axis=0,ascending=True)
# ASPIRATIONS
df['SA']=df.groupby(['Year']).ROE.transform('mean')
df['distance'] = df['ROE'] - df['SA']
df.loc[df.distance<0,'BSA']=df['distance']
df.loc[(df.distance>0)|(df.distance==0),'BSA']=0
df.loc[(df.distance>0)|(df.distance==0),'ASA']=df['distance']
df.loc[df.distance<0,'ASA']=0
# DURATION
df['DURATION'] = 0
mylist = []
for firm in range(df.Id.nunique()):
  mylist.append(df.loc[df.Id==firm+1,['Id','Year','BSA','DURATION']].values.tolist())
m = [0]
for firm in range(df.Id.nunique()):
  for i in mylist[firm]: 
    if i[2]<0:
      m.append([i[0],i[1],m[-1][2]+1])
    else:
      m.append([i[0],i[1],i[3]])
del m[0]
print(len(m))
mydf = pandas.DataFrame(m,columns=['Id','Year','DURATION'])
df = df.drop('DURATION',axis=1)
df = pandas.merge(left=df,right=mydf,how='inner',on=['Year','Id'])
print(df.loc[(df.Id==38)|(df.Id==39),['Id','Year','BSA','DURATION']])
df = df.sort_values(by=['Id','Year'], axis=0, ascending=True)
# ATTENTION
filepath = 'D:/DEA/Data/Results_def_01_for_Elio.txt'
data5 = pandas.read_csv(filepath,sep='\t')
data5 = data5.iloc[1:1008,[0,1,25,26,27]]
data5.columns = ['Year','Id','PRODUCT','GEOGRAPHY','GROWTH']
data5['Year'] = pandas.to_numeric(data5.Year.astype(str).str[0:4])
print(data5)
df = pandas.merge(left=df,right=data5,how='inner',on=['Year','Id'])
df['ATTENTION'] = df[['PRODUCT','GEOGRAPHY','GROWTH']].std(axis=1)
# AMBIDEXTERITY
print(df.Non_RD_alliances.corr(df.RD_alliances),'correlation between RD alliances and non-RD alliances')
df['AMB0'] = df.RD_alliances + df.Non_RD_alliances
df['AMB1'] = df.RD_alliances - df.Non_RD_alliances
df['AMB1'] = df['AMB1'].abs()
# SIZE
df['Size'] = numpy.log(df.Employees) 
df = df.sort_values(by=['Id','Year'], axis=0, ascending=True)
df[17:30]

1232
     Id  Year       BSA  DURATION
814  38  1991  0.000000       0.0
815  38  1992  0.000000       0.0
816  38  1993  0.000000       0.0
817  38  1994  0.000000       0.0
818  38  1995 -0.014391       1.0
819  38  1996 -0.098000       2.0
820  38  1997  0.000000       0.0
821  38  1998 -0.209075       1.0
822  38  1999 -0.059585       2.0
823  38  2000 -0.124833       3.0
824  38  2001 -0.008870       4.0
825  38  2002  0.000000       0.0
826  38  2003  0.000000       0.0
827  38  2004 -0.045434       1.0
828  38  2005 -0.000642       2.0
829  38  2006 -0.000154       3.0
830  38  2007 -0.634098       4.0
831  38  2008 -0.259360       5.0
832  38  2009 -0.163826       6.0
833  38  2010 -0.089444       7.0
834  38  2011 -0.104333       8.0
835  38  2012 -0.025738       9.0
836  39  1991       NaN       0.0
837  39  1992       NaN       0.0
838  39  1993       NaN       0.0
839  39  1994       NaN       0.0
840  39  1995       NaN       0.0
841  39  1996 -0.111000       1.0
842  39  

,Id,Year,Age,Employees,Total_assets,Net_sales,Human_resources_1,Human_resources_2,Physical_resources_1,Physical_resources_2,...,BSA,ASA,DURATION,PRODUCT,GEOGRAPHY,GROWTH,ATTENTION,AMB0,AMB1,Size
17,2,1995,107.0,50241.0,9412.580,10012.194,0.199,5.018,4249.538,0.424,...,0.000000,0.203609,0.0,0.0,0.339996,0.660004,0.330052,15.0,3.0,10.824587
18,2,1996,108.0,52817.0,11125.600,11013.460,0.209,4.796,4461.543,0.405,...,0.000000,0.196000,0.0,0.0,0.407071,0.592929,0.303264,25.0,1.0,10.874588
19,2,1997,109.0,54487.0,12061.068,11883.462,0.218,4.585,4569.691,0.385,...,0.000000,0.289423,0.0,0.0,0.440513,0.559487,0.294741,33.0,13.0,10.905717
20,2,1998,110.0,56236.0,13216.213,12477.845,0.222,4.507,4738.843,0.380,...,0.000000,0.089925,0.0,0.0,0.430937,0.569063,0.296821,23.0,1.0,10.937312
21,2,1999,111.0,57100.0,14471.044,13177.625,0.231,4.333,4770.059,0.362,...,0.000000,0.141415,0.0,0.0,0.435399,0.564601,0.295815,32.0,6.0,10.952559
22,2,2000,112.0,60571.0,15283.254,13745.916,0.227,4.406,4816.911,0.350,...,0.000000,0.167167,0.0,0.0,0.448690,0.551310,0.293200,29.0,21.0,11.011572
23,2,2001,113.0,71426.0,23296.423,16285.246,0.228,4.386,5551.547,0.341,...,-0.023870,0.000000,1.0,0.0,0.299710,0.700290,0.351354,27.0,15.0,11.176417
24,2,2002,114.0,71819.0,24259.102,17684.663,0.246,4.061,5828.122,0.330,...,0.000000,0.121444,0.0,0.0,0.534176,0.465824,0.290691,14.0,10.0,11.181904
25,2,2003,115.0,72181.0,26715.342,19680.561,0.273,3.668,6281.806,0.319,...,0.000000,0.058906,0.0,0.0,0.504841,0.495159,0.288716,14.0,4.0,11.186932
26,2,2004,116.0,60600.0,28767.494,19680.016,0.325,3.079,6007.874,0.305,...,0.000000,0.070566,0.0,0.0,0.479917,0.520083,0.289373,30.0,12.0,11.012050


### Correlation

In [13]:
mydata = df[['AMB0','AMB1','BSA','DURATION','ATTENTION','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
print(len(mydata),'Number of observations')
ds_mean = mydata.agg(['mean','std']).T
corr = mydata.corr()
corr = pandas.concat([ds_mean,corr], axis = 1)
corr = corr.applymap('{:.2f}'.format)
corr['statistics'] = ' '
corr.set_index('statistics',append=True,inplace=True)
print(corr)

def pearsonr_pval(x,y):
  return pearsonr(x,y)[1].round(3)
pval = mydata.corr(method=pearsonr_pval)
pval = pval.applymap('{:.3f}'.format)
pval = pval.applymap(lambda x: '(' + x + ')')
pval['statistics'] = 'p-value'
pval.set_index('statistics',append=True,inplace=True)

corr = pandas.concat([corr,pval],axis=0).sort_index(kind='merge').reindex(['AMB0','AMB1','BSA','DURATION','ATTENTION','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year'],axis=0,level=0)
filepath = 'D:/DEA/DEA-2/Correlation-1.txt'
corr.to_csv(filepath,sep = '\t',header=False,index=False)
corr

1007 Number of observations
                                   mean    std   AMB0   AMB1    BSA DURATION  \
                    statistics                                                 
AMB0                              11.95  11.91   1.00   0.82   0.14    -0.21   
AMB1                               5.07   5.87   0.82   1.00   0.13    -0.17   
BSA                               -0.10   0.21   0.14   0.13   1.00    -0.19   
DURATION                           3.67   6.52  -0.21  -0.17  -0.19     1.00   
ATTENTION                          0.46   0.10   0.05   0.08   0.03    -0.05   
ASA                                0.10   0.33   0.05   0.05   0.14    -0.18   
Absorbed_slack                     1.97  37.88  -0.04  -0.04  -0.06    -0.01   
Unabsorbed_slack                   1.39   1.99  -0.20  -0.16  -0.16     0.06   
Absorptive_capacity                0.23   0.72  -0.08  -0.06  -0.46     0.06   
Market_commonality                 0.41   0.14   0.03   0.01  -0.08     0.03   
Size        

mean    std     AMB0     AMB1      BSA  \
                    statistics                                              
AMB0                              11.95  11.91     1.00     0.82     0.14   
                    p-value         NaN    NaN  (1.000)  (0.000)  (0.000)   
AMB1                               5.07   5.87     0.82     1.00     0.13   
                    p-value         NaN    NaN  (0.000)  (1.000)  (0.000)   
BSA                               -0.10   0.21     0.14     0.13     1.00   
                    p-value         NaN    NaN  (0.000)  (0.000)  (1.000)   
DURATION                           3.67   6.52    -0.21    -0.17    -0.19   
                    p-value         NaN    NaN  (0.000)  (0.000)  (0.000)   
ATTENTION                          0.46   0.10     0.05     0.08     0.03   
                    p-value         NaN    NaN  (0.309)  (0.067)  (0.422)   
ASA                                0.10   0.33     0.05     0.05     0.14   
                    p-value         NaN    NaN  (0.158)  (0.178)  (0.000)   
Absorbed_slack                     1.97  37.88    -0.04    -0.04    -0.06   
                    p-value         NaN    NaN  (0.289)  (0.361)  (0.094)   
Unabsorbed_slack                   1.39   1.99    -0.20    -0.16    -0.16   
                    p-value         NaN    NaN  (0.000)  (0.000)  (0.000)   
Absorptive_capacity                0.23   0.72    -0.08    -0.06    -0.46   
                    p-value         NaN    NaN  (0.024)  (0.079)  (0.000)   
Market_commonality                 0.41   0.14     0.03     0.01    -0.08   
                    p-value         NaN    NaN  (0.459)  (0.769)  (0.014)   
Size                               9.37   1.76     0.41     0.36     0.30   
                    p-value         NaN    NaN  (0.000)  (0.000)  (0.000)   
Year                            2003.51   5.19    -0.19    -0.06     0.04   
                    p-value         NaN    NaN  (0.000)  (0.120)  (0.211)   

                               DURATION ATTENTION      ASA Absorbed_slack  \
                    statistics                                              
AMB0                              -0.21      0.05     0.05          -0.04   
                    p-value     (0.000)   (0.309)  (0.158)        (0.289)   
AMB1                              -0.17      0.08     0.05          -0.04   
                    p-value     (0.000)   (0.067)  (0.178)        (0.361)   
BSA                               -0.19      0.03     0.14          -0.06   
                    p-value     (0.000)   (0.422)  (0.000)        (0.094)   
DURATION                           1.00     -0.05    -0.18          -0.01   
                    p-value     (1.000)   (0.220)  (0.000)        (0.760)   
ATTENTION                         -0.05      1.00    -0.06           0.19   
                    p-value     (0.220)   (1.000)  (0.138)        (0.000)   
ASA                               -0.18     -0.06     1.00          -0.01   
                    p-value     (0.000)   (0.138)  (1.000)        (0.722)   
Absorbed_slack                    -0.01      0.19    -0.01           1.00   
                    p-value     (0.760)   (0.000)  (0.722)        (1.000)   
Unabsorbed_slack                   0.06      0.05     0.10           0.01   
                    p-value     (0.132)   (0.332)  (0.007)        (0.771)   
Absorptive_capacity                0.06      0.10     0.13           0.00   
                    p-value     (0.055)   (0.025)  (0.000)        (0.952)   
Market_commonality                 0.03     -0.06    -0.02          -0.02   
                    p-value     (0.372)   (0.201)  (0.560)        (0.662)   
Size                              -0.16      0.00     0.04          -0.10   
                    p-value     (0.000)   (0.945)  (0.296)        (0.014)   
Year                               0.09     -0.00    -0.03           0.07   
                    p-value     (0.003)   (0.947)  (0.416)        (0.069)   

                             

## Regression

In [14]:
R = []
mydata = df[['Id','AMB0','AMB1','BSA','DURATION','ATTENTION','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
mydata = mydata.dropna()
mydata = mydata/mydata.std()
mydata = mydata.sort_values(by=['Id','Year'], axis=0, ascending=True)
for amb in ['AMB0','AMB1']:
  reg = sm.OLS.from_formula(amb+'~BSA+DURATION+ATTENTION+BSA*ATTENTION+DURATION*ATTENTION+ASA+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+C(Id)+C(Year)',data=mydata).fit(cov_type='hac-panel',cov_kwds={'maxlags':1,'groups':mydata['Id']})   
  R.append((amb,reg.params,reg.bse,reg.pvalues,reg.rsquared_adj))
  print(reg.summary())


                            OLS Regression Results                            
Dep. Variable:                   AMB0   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     36.76
Date:                Thu, 11 Mar 2021   Prob (F-statistic):           3.36e-23
Time:                        11:11:09   Log-Likelihood:                -192.40
No. Observations:                 365   AIC:                             524.8
Df Residuals:                     295   BIC:                             797.8
Df Model:                          69                                         
Covariance Type:            hac-panel                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [15]:
table = Table_Summary(R)
table = table.reindex(['Intercept','BSA','DURATION','ATTENTION','BSA:ATTENTION','DURATION:ATTENTION']+['Adjusted R2'],level=0)
table = table.reindex(['Coeff','CSE','p-val',''],level=1)
table.round(2)

Measures                       AMB0  AMB1
Variables          Statistics            
Intercept          Coeff      -0.54 -1.62
                   CSE         1.42  1.60
                   p-val       0.70  0.31
BSA                Coeff      -0.34 -0.23
                   CSE         0.15  0.22
                   p-val       0.02  0.28
DURATION           Coeff      -0.60 -0.70
                   CSE         0.32  0.36
                   p-val       0.06  0.05
ATTENTION          Coeff      -0.04  0.00
                   CSE         0.12  0.13
                   p-val       0.75  0.99
BSA:ATTENTION      Coeff       0.07  0.04
                   CSE         0.03  0.04
                   p-val       0.02  0.30
DURATION:ATTENTION Coeff       0.09  0.12
                   CSE         0.07  0.07
                   p-val       0.21  0.08
Adjusted R2                    0.79  0.59

In [16]:
table.to_excel('Table_Analyses-1.xlsx',float_format="%.3f")